# ANBW Model Comparsion

## Comparsion Model
- Dacon Baseline Model
- EfficientNet B0
- EfficientNet B1
- EfficientNet B2

### Setup

In [1]:
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = "3"

In [2]:
import gc
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

In [3]:
print("CUDA avilable:",
     True if tf.config.list_physical_devices("GPU") else False)

CUDA avilable: True


In [4]:
class Dataset:
    def __init__(self, path, fraction, resize = None):
        self.path = path
        self.fraction = fraction
        self.resize = resize
        self.SEED = int(np.random.rand() * 10000)
        self.show_info()
    
    def show_info(self):
        print("========================================")
        print(f"path: {self.path}")
        print(f"fraction: {self.fraction}")
        print(f"resize: {self.resize}")
        print(f"SEED: {self.SEED}")
        print("========================================")
    
    def read_csv(self):
        if self.path == None or os.path.isfile(self.path) == False:
            raise Exception(f"Invalid path: {self.path}")
        train = pd.read_csv(f"{self.path}")
        return train
    
    def reshape(self, raw_data):
        features = raw_data.drop(["id", "digit", "letter"], axis=1).values
        features = features.reshape(-1, 28, 28, 1)
        features = np.where((features<=20)&(features!=0), 0., features)
        features = features / 255
        features = features.astype("float32")
        return features
    
    def resize(self, images):
        resized_images = np.zeros([len(images), self.reshape, self.reshape, 3], dtype=np.float32)
        
        for id, image in enumerate(images):
            converted = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
            resized = cv2.resize(converted, (self.resize, self.resize), interpolation=cv2.INTER_CUBIC)
            del converted
            resized_images[id] = resized
            del resized
            gd.collect()
        
        return resized_images
    
    def label_to_vector(self, raw_data):
        labels = raw_data["digit"]
        labels_vector = np.zeros((len(labels), len(labels.unique())))
        
        for id, digit in enumerate(labels):
            labels_vector[id, digit] = 1
        
        return labels_vector
    
    def split(self, x_train, y_train):
        x_splited_train, x_splited_val, y_splited_train, y_splited_val = train_test_split(x_train, y_train, test_size=self.fraction, random_state=self.SEED, shuffle=True)
        return x_splited_train, x_splited_val, y_splited_train, y_splited_val
    
    def preprocess(self):
        train = self.read_csv()
        train_x = self.reshape(train)
        train_y = self.label_to_vector(train)
        if self.resize != None:
            train_x = self.resize(train_x)
        train_x, val_x, train_y, val_y = self.split(train_x, train_y)
        
        return train_x, val_x, train_y, val_y

In [5]:
class Model:
    def baseline_model(input_size):
        inputs = tf.keras.layers.Input(input_size)

        bn = tf.keras.layers.BatchNormalization()(inputs)
        conv = tf.keras.layers.Conv2D(128, kernel_size=5, strides=1, padding='same', activation='relu')(bn)
        bn = tf.keras.layers.BatchNormalization()(conv)
        conv = tf.keras.layers.Conv2D(128, kernel_size=2, strides=1, padding='same', activation='relu')(bn)
        pool = tf.keras.layers.MaxPooling2D((2, 2))(conv)

        bn = tf.keras.layers.BatchNormalization()(pool)
        conv = tf.keras.layers.Conv2D(256, kernel_size=2, strides=1, padding='same', activation='relu')(bn)
        bn = tf.keras.layers.BatchNormalization()(conv)
        conv = tf.keras.layers.Conv2D(256, kernel_size=2, strides=1, padding='same', activation='relu')(bn)
        pool = tf.keras.layers.MaxPooling2D((2, 2))(conv)

        flatten = tf.keras.layers.Flatten()(pool)

        bn = tf.keras.layers.BatchNormalization()(flatten)
        dense = tf.keras.layers.Dense(1000, activation='relu')(bn)

        bn = tf.keras.layers.BatchNormalization()(dense)
        outputs = tf.keras.layers.Dense(10, activation='softmax')(bn)

        model = tf.keras.models.Model(inputs=inputs, outputs=outputs)

        return model
    
    def efficientnet_b0(input_size):
        effnet = tf.keras.applications.EfficientNetB0(
            include_top=True,
            weights=None,
            input_shape=(input_size, input_size, 3),
            classes=10,
            classifier_activation="softmax",
        )
        model = tf.keras.Sequential()
        model.add(effnet)
        
        return model
    
    def efficientnet_b1(input_size):
        effnet = tf.keras.applications.EfficientNetB1(
            include_top=True,
            weights=None,
            input_shape=(input_size, input_size, 3),
            classes=10,
            classifier_activation="softmax",
        )
        model = tf.keras.Sequential()
        model.add(effnet)
        
        return model
    
    def efficientnet_b2(input_size):
        effnet = tf.keras.applications.EfficientNetB2(
            include_top=True,
            weights=None,
            input_shape=(input_size, input_size, 3),
            classes=10,
            classifier_activation="softmax",
        )
        model = tf.keras.Sequential()
        model.add(effnet)
        
        return model

In [6]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [7]:
class Train:
    def __init__(self, x_train, y_train, x_val, y_val, model, epoch, batch, loss, optimizer, metrics, save_path):
        self.x_train = x_train
        self.y_train = y_train
        self.x_val = x_val
        self.y_val = y_val
        self.model = model
        self.batch = batch
        self.epoch = epoch
        self.callbacks = self.callback()
        self.loss = loss
        self.optimizer = optimizer
        self.metrics = metrics
        self.save_path = save_path
        
    def callback(self):
        model_save_path = self.save_path + "ckp-epoch-{epoch:02d}.h5"
        model_checkpoint_callback = ModelCheckpoint(
            filepath=model_save_path,
            monitor="val_accuracy",
            save_best_only=True,
            save_freq=5,
            verbose=1
        )

        early_stopping = EarlyStopping(monitor="val_loss", verbose=1, patience=10),

        reduce_LR = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=1,
                              min_lr=0.00001, verbose=1, mode="min")
        
        return model_checkpoint_callback, early_stopping, reduce_LR
    
    def run(self):
        model = self.model
        model.compile(loss=self.loss, optimizer=self.optimizer, metrics=[self.metrics])
        model.summary()
        
        history = model.fit(self.x_train, self.y_train, batch_size=self.batch, epochs=self.epoch, callbacks=self.callbacks, validation_data=(self.x_valid, self.y_valid))
        
        return history

In [8]:
datast = Dataset("dataset/train.csv", 0.2)
train_x, val_x, train_y, val_y = datast.preprocess()

path: dataset/train.csv
fraction: 0.2
resize: None
SEED: 3788
